<a href="https://colab.research.google.com/github/mitsu666/DataScienceLecture/blob/master/DataScienceLecture5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. テーブルデータの読み込み

In [0]:
#必要なライブラリをインポート
import pandas as pd
#csvから読み込む
df = pd.read_csv('sales201907.csv')
#tsvから読み込む
df = pd.read_table('sales201907.tsv')

#tsvをread_csvで読み込む
df = pd.read_csv('sales201907', sep='\t')

In [0]:
#エンコードがsjisの場合
df = pd.read_csv('sales201907.csv', encoding='sjis')

# 3. データの集約と結合

In [0]:
#3ヶ月分読み込む
df_201907 = pd.read_csv('sales201907.csv')
df_201908 = pd.read_csv('sales201908.csv')
df_201909 = pd.read_csv('sales201909.csv')
#縦に結合
df = pd.concat([df_201907,df_201908,df_201909])
print (df)

In [0]:
pd.pivot_table(df, index='顧客ID', columns='商品名', aggfunc='sum')

In [0]:
#属性データ読み込み
df_zokusei = pd.read_csv('zokusei.csv',engine='python',encoding='cp932')
print (df_zokusei)

In [0]:
#ID単位で集約したテーブルに属性を紐付けます
pd.merge(
    df_zokusei,#属性データ
    pd.pivot_table(df, index='顧客ID', columns='商品名', aggfunc='sum').reset_index(),#顧客IDで紐付けるためindexを解除しておく
    on = '顧客ID',
    how = 'left' #左を優先して結合
)

# 4. テーブルデータの理解

## 4.1 探索的データアナリシス(EDA)

In [0]:
########################
#サンプルデータ  準備
#california housing
########################
from sklearn.datasets import fetch_california_housing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#欠損をランダムに作成するため乱数オブジェクト作成
rng = np.random.RandomState(0)
#california_housing 読み込む
dataset = fetch_california_housing()
#説明変数を取得
X_full = dataset.data
#被説明変数を取得
y_full = dataset.target
#全てのレコードでは大きいので10サンプルごとに抽出
X_full = X_full[::10]
y_full = y_full[::10]
#欠損作成のためにレコード数とカラム数を取得
n_full_samples, n_full_features = X_full.shape
missing_full_samples = np.arange(n_full_samples)
#意図的にレコードの半分を欠損させる
missing_full_samples = random.choice(missing_full_samples,len(missing_full_samples)//2,replace=False)
X_missing = X_full[missing_full_samples].copy()
y = y_full
n_samples, n_features = X_missing.shape
#欠損となる列をランダムに決める
missing_features = rng.choice(n_features, n_samples, replace=True)
#欠損代入
missing_samples = np.arange(n_samples)
X_missing[missing_samples, missing_features] = np.nan
#欠損させない配列
X_not_missing = X_full[setdiff1d(np.arange(n_full_samples), missing_full_samples)].copy()
#欠損させる配列とさせない配列を結合
X = np.concatenate([X_not_missing,X_missing])
#データフレーム化
df_california = pd.DataFrame(np.concatenate([X,y.reshape(-1,1)],axis=1))
df_california.columns=dataset.feature_names+['Target']

########################
#サンプルデータ  準備
#Adult Data Set
########################
df_adult = pd.read_csv('adult.data',header=None)
#変数名付与
df_adult.columns = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]


## 4.2 テーブル全体の理解

In [0]:
#テーブルのサイズ表示
print (df_california.shape)
#カラムごとの欠損ではないレコード数 とデータ型
print (df_california.info())

In [2]:
#カラムごと欠損数
print (df_california.isnull().sum())

NameError: ignored

## 4.3 数値変数の分布

In [0]:
#統計量算出
df_california.describe()

In [0]:
#変数のボックスプロット図を描く
plt.figure(figsize=(14,10))
for i in np.arange((df_california.shape[1])):
    plt.subplot(3,3,i+1)
    plt.boxplot(df_california.dropna().iloc[:,i])
    plt.title(df_california.columns[i])

In [0]:
#変数のヒストグラムを描く
plt.figure(figsize=(14,10))
for i in np.arange((df_california.shape[1])):
    plt.subplot(3,3,i+1)
    plt.hist(df_california.dropna().iloc[:,i])
    plt.title(df_california.columns[i])

## 4.4 カテゴリカル変数の分布

In [0]:
#カテゴリカル変数の要約を確認
df_adult.describe(include='O')
#数値・カテゴリ変数同時に行う
df_adult.describe(include='all')

In [5]:
#プロットエリア定義
plt.figure(figsize=(20,20))
#カテゴリカル変数でヒストグラムを書く
for i,j in enumerate(df_adult.describe(include='O').columns):
    plt.subplot(3,3,i+1)
    plt.barh(width=df_adult.loc[:,j].value_counts(),
         y=df_adult.loc[:,j].value_counts().index)
    plt.title(j,fontdict={'fontsize': 20})

NameError: ignored

## 4.5 変数間の相関

In [6]:
#ヒートマップを作成
import seaborn as sns
plt.figure(figsize=(10,10))
sns.heatmap(df_california.corr(),cmap="YlGnBu",linewidths=5)

NameError: ignored

# 5. カテゴリカル変数の処理

## 5.1 順序ラベル・エンコーディング

In [9]:
!pip install category_encoders 

     |████████████████████████████████| 102kB 2.4MB/s 


In [7]:
#コーヒーサイズデータフレーム作成
df = pd.DataFrame({'SIZE':['L','M','S','L'],'PRICE':['250','200', \
                                                     '150','250']})
#表示
df

,SIZE,PRICE
0,L,250
1,M,200
2,S,150
3,L,250


In [10]:
#category encoder インポート
import category_encoders as ce
#マップを辞書で学習
mapping = [{'col':'SIZE','mapping': {'S': 0, 'M': 1, 'L': 2}}]
#エンコーダーをインスタンス化
enc = ce.OrdinalEncoder(mapping=mapping)
#マップを学習
enc.fit(df)
#変換
enc.transform(df)

,SIZE,PRICE
0,2,250
1,1,200
2,0,150
3,2,250


## 5.2 ワン・ホット・エンコーディング

In [11]:
#OneHotEncoder 1インスタンス化
#use_cat_names をTrueとすることで変数名が元のカテゴリ名に対応する
encoder = ce.OneHotEncoder(use_cat_names=True)
#学習
encoder.fit(df_adult['MaritalStatus'])
#変換
encoder.transform(df_adult['MaritalStatus'])

NameError: ignored

In [12]:
#df全体に適用
encoder = ce.OneHotEncoder(use_cat_names=True)
#学習
encoder.fit(df_adult)
#変換
encoder.transform(df_adult)

NameError: ignored

In [13]:
#購買トランザクション読み込み
df = pd.read_csv('sales201907_9.csv')
df

FileNotFoundError: ignored

In [14]:
#df全体に適用
encoder = ce.OneHotEncoder(use_cat_names=True)
#学習
encoder.fit(df.drop('購買日時',axis=1))
#変換
df_enc = encoder.transform(df.drop('購買日時',axis=1))

KeyError: ignored

In [15]:
#pivotで集約する
pd.pivot_table(df_enc,index='顧客ID',aggfunc='sum')

NameError: ignored

## 5.3 ターゲット・エンコーディング

In [16]:
#データをトレーニング・テストに分ける
from sklearn.model_selection import train_test_split
#NativeCountry取得
X = df_adult['NativeCountry']
#教師データをOneHotEncodingしておく
encoder = ce.OneHotEncoder(use_cat_names=True)
y = encoder.fit_transform(df_adult['Income']).iloc[:,1]
#データ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

NameError: ignored

In [17]:
#ターゲットエンコーダ インスタンス化
encoder = ce.TargetEncoder()
#トレーニングデータで学習
encoder.fit(X_train,y_train)
#トレーニングを変換
X_train_enc = encoder.transform(X_train)
#テストを変換
X_test_enc = encoder.transform(X_test)

NameError: ignored

In [18]:
#元のデータと比較してみる
pd.concat([X_train,X_train_enc],axis=1).drop_duplicates()

NameError: ignored